#### SQL Analysis with MySQL  

**Description**: Create DB/table, load CSV, run queries for insights (trends, risks, aggregates).  

**Problem**: Need scalable querying for transactional/financial data.  

**Solution**: MySQL for structured analysis; use Python to manage connection & loading.  

**Recommendation**: Queries simulate Xapo Finance tasks (e.g., performance reporting, risk detection).  

#### Imports & .env Load

In [1]:
import os  
from dotenv import load_dotenv  
import pymysql  
import pandas as pd  

load_dotenv()  # Load .env  

MYSQL_HOST = os.getenv('MYSQL_HOST')  
MYSQL_USER = os.getenv('MYSQL_USER')  
MYSQL_PASSWORD = os.getenv('MYSQL_PASSWORD')  
MYSQL_DB = os.getenv('MYSQL_DB')  

print("MySQL config loaded:", bool(MYSQL_HOST and MYSQL_USER and MYSQL_PASSWORD and MYSQL_DB))  

# Function to get connection (reuse this)  
def get_connection(db_name=None):  
    return pymysql.connect(  
        host=MYSQL_HOST,  
        user=MYSQL_USER,  
        password=MYSQL_PASSWORD,  
        database=db_name,  
        charset='utf8mb4',  
        cursorclass=pymysql.cursors.DictCursor  
    )  

MySQL config loaded: True


#### Create Database

In [2]:
print("Creating database...")  

conn = get_connection()  # Connect without DB first  
with conn.cursor() as cursor:  
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {MYSQL_DB}")  
    print(f"Database '{MYSQL_DB}' created or already exists.")  
conn.commit()  
conn.close()  

Creating database...
Database 'crypto_analysis_db' created or already exists.


#### Load CSV into Table

In [3]:
print("Loading CSV into MySQL table...")  

# Load CSV with pandas (handles datatypes)  
df = pd.read_csv('crypto_transactional_data_enriched.csv')  
df['timestamp'] = pd.to_datetime(df['timestamp'])  

# Connect and insert (use to_sql for simplicity)  
from sqlalchemy import create_engine  

engine = create_engine(f"mysql+pymysql://{MYSQL_USER}:{MYSQL_PASSWORD}@{MYSQL_HOST}/{MYSQL_DB}")  

df.to_sql(name='crypto_data', con=engine, if_exists='replace', index=False)  

print(f"Loaded {len(df)} rows into 'crypto_data' table.")  

Loading CSV into MySQL table...
Loaded 732 rows into 'crypto_data' table.


#### Basic Verification Query

In [4]:
print("Verifying data load...")  

conn = get_connection(MYSQL_DB)  
with conn.cursor() as cursor:  
    cursor.execute("SELECT COUNT(*) AS row_count FROM crypto_data")  
    result = cursor.fetchone()  
    print("Row count in table:", result['row_count'])  

    cursor.execute("SELECT * FROM crypto_data LIMIT 5")  
    results = cursor.fetchall()  
    print("First 5 rows:")  
    for row in results:  
        print(row)  
conn.close()  

Verifying data load...
Row count in table: 732
First 5 rows:
{'timestamp': datetime.datetime(2025, 1, 14, 0, 0), 'price': 94456.3493746993, 'volume': 76328330233.31137, 'market_cap': 1870609367804.8225, 'coin': 'Bitcoin', 'transaction_count': 201958, 'avg_txn_fee_usd': 5.35853160876468, 'total_txn_value_usd': 1082198.3266428972, 'daily_return': None}
{'timestamp': datetime.datetime(2025, 1, 15, 0, 0), 'price': 96574.07768799194, 'volume': 51241728228.81719, 'market_cap': 1912603332624.1123, 'coin': 'Bitcoin', 'transaction_count': 226867, 'avg_txn_fee_usd': 4.282331541758937, 'total_txn_value_usd': 971519.7098842248, 'daily_return': 0.0224201795571392}
{'timestamp': datetime.datetime(2025, 1, 16, 0, 0), 'price': 100313.15239828422, 'volume': 57637319739.5434, 'market_cap': 1986831430651.8647, 'coin': 'Bitcoin', 'transaction_count': 211932, 'avg_txn_fee_usd': 1.9698231797938013, 'total_txn_value_usd': 417468.56614006, 'daily_return': 0.038717167171633}
{'timestamp': datetime.datetime(202

#### Advanced SQL Analysis Queries

In [5]:
print("Performing best analysis queries...")  

conn = get_connection(MYSQL_DB)  
queries = {  

    "Monthly Aggregates": """  
        SELECT coin, DATE_FORMAT(timestamp, '%Y-%m') AS month,  
            AVG(price) AS avg_price,  
            SUM(volume) AS total_volume,  
            SUM(total_txn_value_usd) AS total_txn_value,  
            AVG(daily_return) AS avg_return  
        FROM crypto_data  
        GROUP BY coin, month  
        ORDER BY month, coin  
    """,  

    "High Volatility Periods": """  
        SELECT coin, timestamp, daily_return,  
            AVG(daily_return) OVER (PARTITION BY coin ORDER BY timestamp ROWS BETWEEN 13 PRECEDING AND CURRENT ROW) AS rolling_vol_14d  
        FROM crypto_data  
        WHERE daily_return IS NOT NULL  
        ORDER BY rolling_vol_14d DESC  
        LIMIT 10  
    """,  

    "Risk Metrics": """  
        SELECT coin,  
            MIN(daily_return) AS worst_day_return,  
            MAX(daily_return) AS best_day_return,  
            STDDEV(daily_return) AS return_std,  
            SUM(CASE WHEN daily_return < 0 THEN 1 ELSE 0 END) / COUNT(*) * 100 AS pct_negative_days  
        FROM crypto_data  
        WHERE daily_return IS NOT NULL  
        GROUP BY coin  
    """,  

    "Volume-Return Correlation Proxy": """  
        SELECT coin,  
            SUM(volume * daily_return) / (COUNT(*) * STDDEV(volume) * STDDEV(daily_return)) AS corr_volume_return  
        FROM crypto_data  
        WHERE daily_return IS NOT NULL  
        GROUP BY coin  
    """,  

    "Top Transaction Days": """  
        SELECT timestamp, coin, total_txn_value_usd, price, volume  
        FROM crypto_data  
        ORDER BY total_txn_value_usd DESC  
        LIMIT 10  
    """  
}  

# Run and print results  
for name, query in queries.items():  
    print(f"\n--- {name} ---")  
    with conn.cursor() as cursor:  
        cursor.execute(query)  
        results = cursor.fetchall()  
        if results:  
            df_query = pd.DataFrame(results)  
            print(df_query.to_string(index=False))  
        else:  
            print("No results.")  
conn.close()  

Performing best analysis queries...

--- Monthly Aggregates ---
    coin   month     avg_price  total_volume  total_txn_value  avg_return
 Bitcoin 2025-01 102266.814044  1.123188e+12     3.267908e+07    0.006343
Ethereum 2025-01   3263.683591  5.256387e+11     4.001106e+07    0.002657
 Bitcoin 2025-02  95922.849136  1.208236e+12     6.910406e+07   -0.007348
Ethereum 2025-02   2706.110842  8.239884e+11     4.834528e+07   -0.011467
 Bitcoin 2025-03  85158.743788  1.076548e+12     6.735879e+07   -0.000323
Ethereum 2025-03   2030.277363  5.650367e+11     6.809344e+07   -0.006702
 Bitcoin 2025-04  86068.125203  1.011116e+12     5.253233e+07    0.004921
Ethereum 2025-04   1687.619294  5.045494e+11     6.516496e+07    0.001007
 Bitcoin 2025-05 103284.770614  1.046452e+12     5.980517e+07    0.003340
Ethereum 2025-05   2352.092648  7.142325e+11     7.265656e+07    0.012127
 Bitcoin 2025-06 105714.032371  8.525042e+11     7.359305e+07    0.001513
Ethereum 2025-06   2516.505031  5.594924e+11    

#### Insights & Recommendations

### Recommendations for Finance & Treasury

1. **Risk & Hedging Strategy**  
   Prioritize tighter controls on Ethereum exposure (volatility ~1.8× Bitcoin). Set automated alerts when 14-day rolling volatility > 0.035–0.04.

2. **Performance & Regulatory Reporting**  
   Use monthly aggregates (avg_price, total_volume, total_txn_value_usd) as core KPIs in Looker dashboards. Include per-coin volatility and negative-day percentage for risk-adjusted performance views.

3. **Liquidity & Revenue Monitoring**  
   Monitor top transaction days and volume surges for liquidity planning and fee revenue forecasting. Even weak volume-return correlation suggests volume is a better indicator of operational activity than directional bets.

4. **Portfolio & Customer Insights**  
   Educate customers on Ethereum’s higher risk/reward profile vs Bitcoin. Recommend diversified holdings or volatility-based rebalancing strategies.

5. **Future Enhancements**  
   - Add indexes on `timestamp` and `coin` for faster queries.  
   - Integrate external data (e.g., BTC dominance, on-chain metrics) via additional tables.  
   - Build stored procedures for recurring risk reports.

**Overall takeaway**: This analysis highlights Ethereum as the higher-risk/higher-reward asset and Bitcoin as more stable — valuable for treasury allocation, risk management, and customer advisory in a crypto banking environment.